# Data Wrangling

For this project, I'll be analyzing data from two datasets. The first is the 'Equine Death and Breakdown' dataset provided by the New York State Gaming Commission. The second is a dataset I compiled myself through my own research on various trainers whether they have a history of drugging their horses. I created this dataset by researching which trainers have been suspended or fined after one or more of their horses had been drug tested and found with traces of drugs over the legal limit (article sources provided in dataset).

For a complete dataset that I will use for my project, I will merge the two datasets together on the trainer names. I will also expand upon different variables in the 'Weather Conditions' and 'Incident Description' columns in order to further test whether those factors are significant.

In [1413]:
# import packages
import pandas as pd
import numpy as np

# Equine Death and Breakdown Dataset

Before merging our two datasets, I will first clean the Equine Death and Breakdown dataset and study its contents.

In [1414]:
# load the dataset as a dataframe
df1 = pd.read_csv('Equine_Death_and_Breakdown.csv')
df1.head()

,Year,Incident Date,Incident Type,Track,Inv Location,Racing Type Description,Division,Weather Conditions,Horse,Trainer,Jockey Driver,Incident Description,Death or Injury
0,2009,03/04/2009,EQUINE DEATH,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,,Private Details,JOHN P. TERRANOVA II,,Private Details-Tr. John Terranova-fell on tra...,Euthanasia
1,2009,03/04/2009,ON-TRACK ACCIDENT,Aqueduct Racetrack (NYRA),,Racing,Thoroughbred,,Private Details,JOHN P. TERRANOVA II,,Private Details-Tr. John Terranova-fell fx LF...,
2,2009,03/04/2009,ON-TRACK ACCIDENT,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,,All Bets Off,B E. LEVINE,,All Bets Off-Tr. Bruce Levine-fell over downed...,
3,2009,03/04/2009,ON-TRACK ACCIDENT,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,,Hot Chile Soup,ENRIQUE ARROYO,,Hot Chile Soup-Tr. Enrique Arroyo-fell over do...,
4,2009,03/04/2009,ON-TRACK ACCIDENT,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,,One Dream Union,BRUCE R. BROWN,,One Dream Union-Tr. Bruce Brown-fell over down...,


In [1415]:
# use the info() method to identify size and datatypes of the dataframe
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 13 columns):
Year                       3218 non-null int64
Incident Date              3218 non-null object
Incident Type              3218 non-null object
Track                      3218 non-null object
Inv Location               3216 non-null object
Racing Type Description    3218 non-null object
Division                   3218 non-null object
Weather Conditions         3171 non-null object
Horse                      3218 non-null object
Trainer                    3218 non-null object
Jockey Driver              3218 non-null object
Incident Description       3218 non-null object
Death or Injury            3217 non-null object
dtypes: int64(1), object(12)
memory usage: 326.9+ KB


In [1416]:
# check the column names
df1.columns

Index(['Year', 'Incident Date', 'Incident Type', 'Track', 'Inv Location',
       'Racing Type Description', 'Division', 'Weather Conditions', 'Horse',
       'Trainer', 'Jockey Driver', 'Incident Description', 'Death or Injury'],
      dtype='object')

In [1417]:
# check the shape of the dataframe
df1.shape

(3218, 13)

Our dataframe has 3,218 rows and 13 columns. We can see that there are some columns that are missing data (e.g. 'Weather Conditons' only has 3,171 records). But we can also see by looking at the head of our dataframe that there are other columns that have blanks but are still being considered as full records. We want to keep our dataset consistent between columns, so we will convert the blanks to null values. Let's check all columns of type object and how many blank cells they contain.

In [1418]:
# count how many blank cells there are in each column of type object
print('Incident Date', df1.loc[df1['Incident Date'] == ' ']['Incident Date'].count())
print('Incident Type', df1.loc[df1['Incident Type'] == ' ']['Incident Type'].count())
print('Track', df1.loc[df1['Track'] == ' ']['Track'].count())
print('Inv Location', df1.loc[df1['Inv Location'] == ' ']['Inv Location'].count())
print('Racing Type Description', df1.loc[df1['Racing Type Description'] == ' ']['Racing Type Description'].count())
print('Division', df1.loc[df1['Division'] == ' ']['Division'].count())
print('Weather Conditions', df1.loc[df1['Weather Conditions'] == ' ']['Weather Conditions'].count())
print('Horse', df1.loc[df1['Horse'] == ' ']['Horse'].count())
print('Trainer', df1.loc[df1['Trainer'] == ' ']['Trainer'].count())
print('Jockey Driver', df1.loc[df1['Jockey Driver'] == ' ']['Jockey Driver'].count())
print('Incident Description', df1.loc[df1['Incident Description'] == ' ']['Incident Description'].count())
print('Death or Injury', df1.loc[df1['Death or Injury'] == ' ']['Death or Injury'].count())

Incident Date 0
Incident Type 0
Track 0
Inv Location 101
Racing Type Description 0
Division 0
Weather Conditions 999
Horse 1
Trainer 54
Jockey Driver 1142
Incident Description 0
Death or Injury 478


The 'Inv Location', 'Weather Conditions', 'Horse', 'Trainer', 'Jockey Driver', and 'Death or Injury' columns all contain blank cells. Let's replace these blanks with nulls.

In [1419]:
# use replace() to convert all ' ' cells to null
df1 = df1.replace(' ', np.NaN)

# count how many blank cells there are in each column of type object
print('Incident Type', df1.loc[df1['Incident Type'] == ' ']['Incident Type'].count())
print('Track', df1.loc[df1['Track'] == ' ']['Track'].count())
print('Inv Location', df1.loc[df1['Inv Location'] == ' ']['Inv Location'].count())
print('Racing Type Description', df1.loc[df1['Racing Type Description'] == ' ']['Racing Type Description'].count())
print('Division', df1.loc[df1['Division'] == ' ']['Division'].count())
print('Weather Conditions', df1.loc[df1['Weather Conditions'] == ' ']['Weather Conditions'].count())
print('Horse', df1.loc[df1['Horse'] == ' ']['Horse'].count())
print('Trainer', df1.loc[df1['Trainer'] == ' ']['Trainer'].count())
print('Jockey Driver', df1.loc[df1['Jockey Driver'] == ' ']['Jockey Driver'].count())
print('Incident Description', df1.loc[df1['Incident Description'] == ' ']['Incident Description'].count())
print('Death or Injury', df1.loc[df1['Death or Injury'] == ' ']['Death or Injury'].count())

Incident Type 0
Track 0
Inv Location 0
Racing Type Description 0
Division 0
Weather Conditions 0
Horse 0
Trainer 0
Jockey Driver 0
Incident Description 0
Death or Injury 0


In [1420]:
# check how non-null records exist for each column
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 13 columns):
Year                       3218 non-null int64
Incident Date              3218 non-null object
Incident Type              3218 non-null object
Track                      3218 non-null object
Inv Location               3115 non-null object
Racing Type Description    3218 non-null object
Division                   3218 non-null object
Weather Conditions         2172 non-null object
Horse                      3217 non-null object
Trainer                    3164 non-null object
Jockey Driver              2076 non-null object
Incident Description       3218 non-null object
Death or Injury            2739 non-null object
dtypes: int64(1), object(12)
memory usage: 326.9+ KB


We've successfully converted all our blank cells to nulls.

Since most of the data is categorical, let's check to see that the categories in various columns aren't mispelled and creating more values than necessary.

In [1421]:
# use groupby() to display the different incident types
# use count() to display how many records for each incident type
df1.groupby(['Incident Type'])['Incident Type'].count()

Incident Type
ACCIDENT - DRIVER/JOCKEY               10
ACCIDENT - IN STARTING GATE            22
ACCIDENT - ON TRACK                   107
ACCIDENT - TAGGED SULKY                56
DRIVER/JOCKEY INJURED                   7
EQUINE DEATH                         1219
EQUINE DEATH - INFECTIOUS DISEASE      10
FALL OF HORSE                         113
FALL OF RIDER                         211
ON-TRACK ACCIDENT                       7
RACING INJURY                         302
STEWARDS/VETS LIST                   1154
Name: Incident Type, dtype: int64

There are 12 incident types, each unique except there are two categories related to on track accidents. They are the 'ACCIDENT - ON TRACK' and 'ON-TRACK ACCIDENT' categories. Let's recategorize the rows with the second description to be categorized as the first.

In [1422]:
# recategorize the on track accidents so they have the same incident type name
df1['Incident Type'] = df1['Incident Type'].replace('ON-TRACK ACCIDENT','ACCIDENT - ON TRACK')
df1.groupby(['Incident Type'])['Incident Type'].count()

Incident Type
ACCIDENT - DRIVER/JOCKEY               10
ACCIDENT - IN STARTING GATE            22
ACCIDENT - ON TRACK                   114
ACCIDENT - TAGGED SULKY                56
DRIVER/JOCKEY INJURED                   7
EQUINE DEATH                         1219
EQUINE DEATH - INFECTIOUS DISEASE      10
FALL OF HORSE                         113
FALL OF RIDER                         211
RACING INJURY                         302
STEWARDS/VETS LIST                   1154
Name: Incident Type, dtype: int64

We now have 11 incident types and each is different from the other. Now let's change the datatype from an object to categorical.

In [1423]:
# update the datatype of 'Incident Type' to be categorical and check the datatype
df1['Incident Type'] = df1['Incident Type'].astype('category')
df1['Incident Type'].dtype

category

Let's move on to check some other columns that are also categorical in nature.

In [1424]:
# check the track names
df1.groupby(['Track'])['Track'].count()

Track
Aqueduct Racetrack (NYRA)               619
Batavia Downs                            63
Belmont Park (NYRA)                     719
Buffalo Raceway                         101
Finger Lakes Gaming & Racetrack         508
Monticello Raceway & Mighty M Gaming    203
Saratoga Gaming & Raceway               400
Saratoga Racecourse (NYRA)              326
Tioga Downs                              80
Vernon Downs                             62
Yonkers Raceway                         137
Name: Track, dtype: int64

In [1425]:
# check racing type descriptions
df1.groupby(['Racing Type Description'])['Racing Type Description'].count()

Racing Type Description
Non-Racing     313
Racing        2512
Training       373
Unknown         20
Name: Racing Type Description, dtype: int64

In [1426]:
# check division
df1.groupby(['Division'])['Division'].count()

Division
Harness         1046
Thoroughbred    2172
Name: Division, dtype: int64

The 'Track', 'Racing Type Description', and 'Division' columns all have good categorical data. Let's convert their data types to cateogorical and then keep checking other columns with categorical data.

In [1427]:
# update dataype of 'Track', 'Racing Type Description', and 'Division to category
df1['Track'] = df1['Track'].astype('category')
df1['Racing Type Description'] = df1['Racing Type Description'].astype('category')
df1['Division'] = df1['Division'].astype('category')

# check datatypes of all the columns by calling the info() method
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 13 columns):
Year                       3218 non-null int64
Incident Date              3218 non-null object
Incident Type              3218 non-null category
Track                      3218 non-null category
Inv Location               3115 non-null object
Racing Type Description    3218 non-null category
Division                   3218 non-null category
Weather Conditions         2172 non-null object
Horse                      3217 non-null object
Trainer                    3164 non-null object
Jockey Driver              2076 non-null object
Incident Description       3218 non-null object
Death or Injury            2739 non-null object
dtypes: category(4), int64(1), object(8)
memory usage: 240.0+ KB


Four of our columns now have data of type category. Let's check the 'Death or Injury' column.

In [1428]:
# check death or injury
df1.groupby(['Death or Injury'])['Death or Injury'].count()

Death or Injury
Accident                        249
Death                             3
Equine Death                    679
Equine Injury                   101
Equine Injury / Equine Death     46
Euthanasia                      460
Injury                          218
Lame no death                     1
Lameness                        110
Steward's List                  871
death                             1
Name: Death or Injury, dtype: int64

There are a few adjustments we can make to the categories in the 'Death or Injury' column. To start, there are three categories for death: 'Death', 'Equine Death', 'death'. The same is true for categories related to lameness ('Lameness' and 'Lame no death') and related to injury ('Equine Injury' and 'Injury'). We can probably assume that horses marked as lame didn't die at the time of the incident and we can probably assume that 'Injury' is in reference to the horse's injury and not the jockey's injury. We can consolidate all of these.

In [1429]:
# use the replace() method to conslidate categories that are the same
df1['Death or Injury'] = df1['Death or Injury'].replace(['Death','death'],'Equine Death')
df1['Death or Injury'] = df1['Death or Injury'].replace('Lame no death','Lameness')
df1['Death or Injury'] = df1['Death or Injury'].replace('Injury','Equine Injury')
df1.groupby(['Death or Injury'])['Death or Injury'].count()

Death or Injury
Accident                        249
Equine Death                    683
Equine Injury                   319
Equine Injury / Equine Death     46
Euthanasia                      460
Lameness                        111
Steward's List                  871
Name: Death or Injury, dtype: int64

We have a lot of rows with null values. We should check to see how these categories match up with the 'Incident Type' column since the descriptions are fairly similar. If there are blank rows in the 'Death or Injury' column that line up with categoreis in the 'Incident Type' column that suggest the horse died, perhaps we can fill in the blank rows with 'Equine Death.

In [1430]:
# identify matching pairs of 'Incident Type' and 'Death or Injury' in our dataframe
df1[['Incident Type','Death or Injury']].drop_duplicates().sort_values('Incident Type').reset_index(drop=True)

,Incident Type,Death or Injury
0,ACCIDENT - DRIVER/JOCKEY,NaN
1,ACCIDENT - DRIVER/JOCKEY,Accident
2,ACCIDENT - IN STARTING GATE,NaN
3,ACCIDENT - IN STARTING GATE,Steward's List
4,ACCIDENT - IN STARTING GATE,Accident
5,ACCIDENT - IN STARTING GATE,Equine Injury
6,ACCIDENT - ON TRACK,Accident
7,ACCIDENT - ON TRACK,Equine Injury
8,ACCIDENT - ON TRACK,Steward's List
9,ACCIDENT - ON TRACK,NaN


The above table shows up that the 'Incident Type' and 'Death or Injury' categories don't match up very well. There's a lot of overlap between the two columns with a total of 43 pairs. Since the 'Incident Type' column is fully populated, we can use this column for most of our analysis. For now, we will leave the blank rows in 'Death or Injury' as is. We may revisit this later.

Finally, convert this column to datatype category.

In [1431]:
# update the datatype of 'Death or Injury' to be categorical and check the datatype
df1['Death or Injury'] = df1['Death or Injury'].astype('category')
df1['Death or Injury'].dtype

category

We also have a date listed for when the incident happened. This is in the 'Incident Date' column. Right now, the incident date isn't a datetime object. Let's convert it to make it easier to use and analyze later on.

In [1432]:
# update the datatype of 'Incident Date' to be a datetime object and check the datatype
df1['Incident Date'] = pd.to_datetime(df1['Incident Date'],format='%m/%d/%Y')
df1['Incident Date'].dtype

dtype('<M8[ns]')

In [1433]:
# check all datatypes using the info() method
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 13 columns):
Year                       3218 non-null int64
Incident Date              3218 non-null datetime64[ns]
Incident Type              3218 non-null category
Track                      3218 non-null category
Inv Location               3115 non-null object
Racing Type Description    3218 non-null category
Division                   3218 non-null category
Weather Conditions         2172 non-null object
Horse                      3217 non-null object
Trainer                    3164 non-null object
Jockey Driver              2076 non-null object
Incident Description       3218 non-null object
Death or Injury            2739 non-null category
dtypes: category(5), datetime64[ns](1), int64(1), object(6)
memory usage: 218.4+ KB


Another variable that I want to test is whether the weather is a significant factor in whether horses get injured or die. We're provided with a 'Weather Conditions' column, but the descriptions aren't all consistent and some descriptions are more elaborate than others. Let's check to see how many unique descriptions there are in the 'Weather Conditions' column.

In [1434]:
# check number of unique strings
df1['Weather Conditions'].nunique()

798

798 different descriptions is a lot, but it makes sense that there are so many since many of them list the temperature and the weather condition (sometimes more than 1 weather condition). Let's see what the top weather condition descriptions are to get a better idea of what the data in this column looks like.

In [1435]:
# display some of the more common weather condition descriptions
df1['Weather Conditions'].value_counts().head(15)

Clear                          306
Cloudy                         125
Sunny                           39
Clear 50 to 55 : degrees F      34
Clear 60 to 65 : degrees F      28
clear                           27
Cloudy 50 to 55 : degrees F     24
Rain                            22
Overcast                        22
80* Clear                       20
Clear 30 to 40 : degrees F      18
Clear 45 to 50 : degrees F      18
60* Clear                       17
Clear 75 to 80 : degrees F      16
75* Clear                       16
Name: Weather Conditions, dtype: int64

A good chunk of our records don't have anything recorded for the weather conditions. We can also see that there is at least one inconsistency with capitalization (clear and Clear). Instead of altering the data within this column, I'll create a few new columns filled with Boolean data. If a weather condition keyword appears in the weather condition description, the result will be 'True', else it will be 'False'. If the 'Weather Conditions' column is blank, all Boolean weather columns will remain blank. Overall, this will be a useful way to display the data since we have overlap between some weather conditions.

Column by column, I will identify whether a key condition word exists in the weather condition description. If it does exist, the boolean value will be set to True for that record in the corresponding weather condition column. Otherwise, the value will be set to False.

In [1436]:
# create new boolean columns
df1['Cloudy'] = np.nan
df1['Sunny'] = np.nan
df1['Clear'] = np.nan
df1['Overcast'] = np.nan
df1['Rain'] = np.nan
df1['Snow'] = np.nan
df1['Wind'] = np.nan
df1['Thunder Storm'] = np.nan
df1['Hot'] = np.nan
df1['Humid'] = np.nan
df1['Warm'] = np.nan

# use info() to display all columns
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 24 columns):
Year                       3218 non-null int64
Incident Date              3218 non-null datetime64[ns]
Incident Type              3218 non-null category
Track                      3218 non-null category
Inv Location               3115 non-null object
Racing Type Description    3218 non-null category
Division                   3218 non-null category
Weather Conditions         2172 non-null object
Horse                      3217 non-null object
Trainer                    3164 non-null object
Jockey Driver              2076 non-null object
Incident Description       3218 non-null object
Death or Injury            2739 non-null category
Cloudy                     0 non-null float64
Sunny                      0 non-null float64
Clear                      0 non-null float64
Overcast                   0 non-null float64
Rain                       0 non-null float64
Snow                  

In [1437]:
# populate 'Cloudy' column

# create make_cloudy function
def make_cloudy(condition):
    if 'cloud' in str(condition).lower():
        return True
    else:
        return False

# apply make_cloudy function
df1['Cloudy'] = df1['Weather Conditions'].apply(make_cloudy)

In [1438]:
# populate 'Sunny' column

# create make_sunny function
def make_sunny(condition):
    if 'sun' in str(condition).lower():
        return True
    else:
        return False

# apply make_sunny function
df1['Sunny'] = df1['Weather Conditions'].apply(make_sunny)

In [1439]:
# populate 'Clear' column

# create make_clear function
def make_clear(condition):
    if 'clear' in str(condition).lower():
        return True
    else:
        return False

# apply make_clear function
df1['Clear'] = df1['Weather Conditions'].apply(make_clear)

In [1440]:
# populate 'Overcast' column

# create make_overcast function
def make_overcast(condition):
    if 'overcast' in str(condition).lower():
        return True
    else:
        return False

# apply make_overcast function
df1['Overcast'] = df1['Weather Conditions'].apply(make_overcast)

In [1441]:
# populate 'Rain' column

# create make_rain function
def make_rain(condition):
    if 'rain' in str(condition).lower():
        return True
    else:
        return False
    
# apply make_rain function
df1['Rain'] = df1['Weather Conditions'].apply(make_rain)

In [1442]:
# populate 'Snow' column

# create make_snow function
def make_snow(condition):
    if 'snow' in str(condition).lower():
        return True
    else:
        return False
    
# apply make_snow function
df1['Snow'] = df1['Weather Conditions'].apply(make_snow)

In [1443]:
# populate 'Wind' column

# create make_wind function
def make_wind(condition):
    if 'wind' in str(condition).lower():
        return True
    else:
        return False
    
# apply make_wind function
df1['Wind'] = df1['Weather Conditions'].apply(make_wind)

In [1444]:
# populate 'Thunder Storm' column

# create make_thunder function
def make_thunder(condition):
    if 'thunder' in str(condition).lower():
        return True
    else:
        return False
    
# apply make_thunder function
df1['Thunder Storm'] = df1['Weather Conditions'].apply(make_thunder)

In [1445]:
# populate 'Hot' column

# create make_hot function
def make_hot(condition):
    if 'hot' in str(condition).lower():
        return True
    else:
        return False
    
# apply make_hot function
df1['Hot'] = df1['Weather Conditions'].apply(make_hot)

In [1446]:
# populate 'Humid' column

# create make_humid function
def make_humid(condition):
    if 'humid' in str(condition).lower():
        return True
    else:
        return False
    
# apply make_humid function
df1['Humid'] = df1['Weather Conditions'].apply(make_humid)

In [1447]:
# populate 'Warm' column

#create make_warm function
def make_warm(condition):
    if 'warm' in str(condition).lower():
        return True
    else:
        return False
    
# apply make_warm function
df1['Warm'] = df1['Weather Conditions'].apply(make_warm)

Now that we've populated the boolean weather columns, let's check the contents of the final dataframe.

In [1448]:
# display the head of the dataframe
df1.head(15)

,Year,Incident Date,Incident Type,Track,Inv Location,Racing Type Description,Division,Weather Conditions,Horse,Trainer,...,Sunny,Clear,Overcast,Rain,Snow,Wind,Thunder Storm,Hot,Humid,Warm
0,2009,2009-03-04,EQUINE DEATH,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,NaN,Private Details,JOHN P. TERRANOVA II,...,False,False,False,False,False,False,False,False,False,False
1,2009,2009-03-04,ACCIDENT - ON TRACK,Aqueduct Racetrack (NYRA),NaN,Racing,Thoroughbred,NaN,Private Details,JOHN P. TERRANOVA II,...,False,False,False,False,False,False,False,False,False,False
2,2009,2009-03-04,ACCIDENT - ON TRACK,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,NaN,All Bets Off,B E. LEVINE,...,False,False,False,False,False,False,False,False,False,False
3,2009,2009-03-04,ACCIDENT - ON TRACK,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,NaN,Hot Chile Soup,ENRIQUE ARROYO,...,False,False,False,False,False,False,False,False,False,False
4,2009,2009-03-04,ACCIDENT - ON TRACK,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,NaN,One Dream Union,BRUCE R. BROWN,...,False,False,False,False,False,False,False,False,False,False
5,2009,2009-03-04,ACCIDENT - ON TRACK,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,NaN,Truly Devine,GARY C. CONTESSA,...,False,False,False,False,False,False,False,False,False,False
6,2009,2009-03-05,EQUINE DEATH,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,Clear & Cold,Sigh You,NaN,...,False,True,False,False,False,False,False,False,False,False
7,2009,2009-03-06,STEWARDS/VETS LIST,Saratoga Gaming & Raceway,NaN,Racing,Harness,NaN,Raid the Bank N,ELAINE M. ELMORE,...,False,False,False,False,False,False,False,False,False,False
8,2009,2009-03-24,STEWARDS/VETS LIST,Monticello Raceway & Mighty M Gaming,MR,Racing,Harness,Cloudy/cold,Grouse A,TODD J. MARCIANO,...,False,False,False,False,False,False,False,False,False,False
9,2009,2009-03-28,STEWARDS/VETS LIST,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,Cloudy,Dr. Silver Packet,JOHN P. CAMPO JR,...,False,False,False,False,False,False,False,False,False,False


In [1449]:
# use info() to display columns
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 24 columns):
Year                       3218 non-null int64
Incident Date              3218 non-null datetime64[ns]
Incident Type              3218 non-null category
Track                      3218 non-null category
Inv Location               3115 non-null object
Racing Type Description    3218 non-null category
Division                   3218 non-null category
Weather Conditions         2172 non-null object
Horse                      3217 non-null object
Trainer                    3164 non-null object
Jockey Driver              2076 non-null object
Incident Description       3218 non-null object
Death or Injury            2739 non-null category
Cloudy                     3218 non-null bool
Sunny                      3218 non-null bool
Clear                      3218 non-null bool
Overcast                   3218 non-null bool
Rain                       3218 non-null bool
Snow                  

In [1450]:
# display new shape of dataframe
df1.shape

(3218, 24)

The Equine Death and Breakdown is now clean enough for us to use. We've standardized our blank/null values to null, consolidated some categories, changed datatypes of some columns, and added boolean columns relating to weather conditions.

# Doping History Trainers Dataset

I compiled this dataset through my own research. Let's review the contents.

In [1451]:
# load the dataset as a dataframe
df2 = pd.read_csv('Doping_History_Trainers.csv',sep=';')
df2.head()

,Trainer,Year of Action,Action Description,Source
0,STEVEN M. ASMUSSEN,2015,fined for violating New York State Gaming Comm...,https://www.nytimes.com/2015/11/24/sports/stev...
1,ANTONIO C. AVILA,2015,suspended and fined for doping a horse,http://www.sandiegouniontribune.com/sports/hor...
2,MICHAEL S. WEINER,2017,fined for racing four horses at Monticello Rac...,http://www.harnesslink.com/News/Trainer-suspen...
3,BOB BAFFERT,2013,unproven but misconduct seems likely,http://www.nytimes.com/2013/11/23/sports/despi...
4,LUIS PENA,2015,banned for three years and fined for more than...,https://www.bloodhorse.com/horse-racing/articl...


In [1452]:
# use the info() method to identify size and datatypes of the dataframe
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
Trainer               24 non-null object
Year of Action        24 non-null int64
Action Description    24 non-null object
Source                24 non-null object
dtypes: int64(1), object(3)
memory usage: 848.0+ bytes


In [1453]:
# check the column names
df2.columns

Index(['Trainer', 'Year of Action', 'Action Description', 'Source'], dtype='object')

In [1454]:
# check the shape of the dataframe
df2.shape

(24, 4)

This is a much smaller dataset than the Equine Death and Breakdown dataset. Since I had compiled this csv file myself, it's already looks the way I would want it to. The datatypes for each column are correct and I don't have any other changes to make.

# Merging the Data

For my final dataset, I want to do a left join between the Equine Death and Breakdown dataframe (df1) and the Doping History Trainers dataframe (df2). The data will be merged on the trainer names, in the column 'Trainer' (same column name in both datasets). To check that the data was properly merged, we'll check whether data for the trainer 'STEVEN M. ASMUSSEN' is properly displayed.

In [1455]:
# merge the two dataframes on 'Trainer'
finaldf = df1.merge(df2,on='Trainer',how='left')
finaldf.head()

,Year,Incident Date,Incident Type,Track,Inv Location,Racing Type Description,Division,Weather Conditions,Horse,Trainer,...,Rain,Snow,Wind,Thunder Storm,Hot,Humid,Warm,Year of Action,Action Description,Source
0,2009,2009-03-04,EQUINE DEATH,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,NaN,Private Details,JOHN P. TERRANOVA II,...,False,False,False,False,False,False,False,NaN,NaN,NaN
1,2009,2009-03-04,ACCIDENT - ON TRACK,Aqueduct Racetrack (NYRA),NaN,Racing,Thoroughbred,NaN,Private Details,JOHN P. TERRANOVA II,...,False,False,False,False,False,False,False,NaN,NaN,NaN
2,2009,2009-03-04,ACCIDENT - ON TRACK,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,NaN,All Bets Off,B E. LEVINE,...,False,False,False,False,False,False,False,NaN,NaN,NaN
3,2009,2009-03-04,ACCIDENT - ON TRACK,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,NaN,Hot Chile Soup,ENRIQUE ARROYO,...,False,False,False,False,False,False,False,NaN,NaN,NaN
4,2009,2009-03-04,ACCIDENT - ON TRACK,Aqueduct Racetrack (NYRA),Aqueduct,Racing,Thoroughbred,NaN,One Dream Union,BRUCE R. BROWN,...,False,False,False,False,False,False,False,NaN,NaN,NaN


In [1456]:
# check the first trainer name that appears in the doping data set to check that it's properly merged
finaldf.loc[finaldf['Trainer'] == 'STEVEN M. ASMUSSEN'].head()

,Year,Incident Date,Incident Type,Track,Inv Location,Racing Type Description,Division,Weather Conditions,Horse,Trainer,...,Rain,Snow,Wind,Thunder Storm,Hot,Humid,Warm,Year of Action,Action Description,Source
34,2009,2009-04-28,EQUINE DEATH,Belmont Park (NYRA),1mile 1/4,Training,Thoroughbred,Clear,Timberah,STEVEN M. ASMUSSEN,...,False,False,False,False,False,False,False,2015.0,fined for violating New York State Gaming Comm...,https://www.nytimes.com/2015/11/24/sports/stev...
207,2009,2009-08-29,STEWARDS/VETS LIST,Saratoga Racecourse (NYRA),5/8 Pole.,Racing,Thoroughbred,Cloudy,Cairiene,STEVEN M. ASMUSSEN,...,False,False,False,False,False,False,False,2015.0,fined for violating New York State Gaming Comm...,https://www.nytimes.com/2015/11/24/sports/stev...
208,2009,2009-08-29,STEWARDS/VETS LIST,Saratoga Racecourse (NYRA),5/8 Pole.,Unknown,Thoroughbred,Cloudy,Cairiene,STEVEN M. ASMUSSEN,...,False,False,False,False,False,False,False,2015.0,fined for violating New York State Gaming Comm...,https://www.nytimes.com/2015/11/24/sports/stev...
339,2010,2010-01-01,FALL OF RIDER,Aqueduct Racetrack (NYRA),At start of race.,Racing,Thoroughbred,Cloudy,Raffie's Deer,STEVEN M. ASMUSSEN,...,False,False,False,False,False,False,False,2015.0,fined for violating New York State Gaming Comm...,https://www.nytimes.com/2015/11/24/sports/stev...
590,2010,2010-08-09,EQUINE DEATH,Belmont Park (NYRA),1/8 pole,Training,Thoroughbred,Clear 83*,Edwards,STEVEN M. ASMUSSEN,...,False,False,False,False,False,False,False,2015.0,fined for violating New York State Gaming Comm...,https://www.nytimes.com/2015/11/24/sports/stev...


In [1457]:
# check the datatypes of all the columns
finaldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3218 entries, 0 to 3217
Data columns (total 27 columns):
Year                       3218 non-null int64
Incident Date              3218 non-null datetime64[ns]
Incident Type              3218 non-null category
Track                      3218 non-null category
Inv Location               3115 non-null object
Racing Type Description    3218 non-null category
Division                   3218 non-null category
Weather Conditions         2172 non-null object
Horse                      3217 non-null object
Trainer                    3164 non-null object
Jockey Driver              2076 non-null object
Incident Description       3218 non-null object
Death or Injury            2739 non-null category
Cloudy                     3218 non-null bool
Sunny                      3218 non-null bool
Clear                      3218 non-null bool
Overcast                   3218 non-null bool
Rain                       3218 non-null bool
Snow                  

It looks like the 'Year of Action' was converted to a float. This occurred because of the empty rows being filled with null values.

Everything seems to be in place. Now we just need to export this to a new csv file.

In [1458]:
# export finaldf to csv file
finaldf.to_csv('Equine_Breakdown_Death_Doping.csv')